### Importación de Librerías

In [1]:
import pyarrow
import fastparquet
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib as hl
import re

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer

from sklearn import (base, decomposition, feature_extraction, impute,
                     neighbors, preprocessing)
from sklearn.preprocessing import MinMaxScaler
from unidecode import unidecode

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

### Lectura del Dataset

In [2]:
df_orig = pd.read_parquet("cupones_ds_test.parquet")
df = df_orig.copy()

In [3]:
df

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,sexo_descripcion,pais,provincia,ciudad,domicilio_codigo_postal,domicilio_barrio,fecha_de_ingreso_histo,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo
0,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,Mujer,Argentina,SALTA,METAN,4440.0,MARCOS AVELLANEDA,2008-10-01,SIN DATOS,SECUNDARIOS,Otros
1,0c138b0842fc5abb7a8c47ffc992e190346,0,030099104,TARJETA NARANJA RIO CUARTO,60001,0,PL,1,3,1,...,Hombre,Argentina,CORDOBA,RIO CUARTO,5800.0,BUENA VISTA,2000-11-20,EMP.SEG.PUBLICA,PRIMARIOS,Propia
2,4c994053b1228fb815332c97f7bd81fa997,0,030099104,TARJETA NARANJA RIO CUARTO,51550,1,PL,1,0,1,...,Hombre,Argentina,CORDOBA,LAS HIGUERAS,5805.0,CENTRO-LAS HIGUERAS,2012-02-15,EMP.SEG.PUBLICA,SECUNDARIOS,Propia
3,daebbfeda20de1dd3167cfc1dd4aa33e339,0,030099168,TARJETA NARANJA RAFAELA,275812,0,PL,1,3,1,...,Mujer,Argentina,SANTA FE,RAFAELA,2300.0,9 DE JULIO,2018-08-13,SIN DATOS,SECUNDARIOS,Propia
4,1eabd3ed6213cf09d37fcfcb212b084f086,0,030099168,TARJETA NARANJA RAFAELA,276289,0,PL,1,3,1,...,Hombre,Argentina,SANTA FE,BELLA ITALIA,2301.0,ZONA URBANA,2014-01-17,ADMINISTRATIVO,SECUNDARIOS,Propia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,38f3775572b0e76646dc34a70ed98a06869,0,078199814,TASI 1 - SAN VICENTE,956922,1,PL,1,0,1,...,Mujer,Argentina,CORDOBA,CORDOBA,5000.0,CORDOBA,2020-08-14,SIN DATOS,SECUNDARIOS,Propia
3765,bc102d66732bb9cd20c43cc6ef2317bd115,0,078199814,TASI 1 - SAN VICENTE,770359,0,PL,1,0,1,...,Mujer,Argentina,CORDOBA,CORDOBA,5000.0,CORDOBA,2016-05-12,SIN DATOS,PRIMARIOS,Propia
3766,69f67fdf30fa648fe95127b2096846a7890,0,078199885,TASI 1 - CORRIENTES,329915,1,PL,1,0,1,...,Mujer,Argentina,CORRIENTES,CORRIENTES,3400.0,JUAN XXIII,2004-05-20,DOCENTE,TERCIARIOS,Propia
3767,7f0963a8769b7c0b3ef034cc3060e8f6901,0,078199885,TASI 1 - CORRIENTES,628152,0,PL,1,3,1,...,Hombre,Argentina,CORRIENTES,RIACHUELO,3416.0,CENTRO,2016-05-11,SIN DATOS,SECUNDARIOS,Propia


In [4]:
df['monto_compra_movimiento'].describe().round(2)

count      3769.00
mean       7505.14
std       16613.69
min      -35609.05
25%          -5.34
50%          40.33
75%        7903.68
max      162175.54
Name: monto_compra_movimiento, dtype: float64

In [5]:
df_orig.shape

(3769, 41)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3769 entries, 0 to 3768
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dni_titular_movimiento               3769 non-null   object 
 1   moneda_movimiento                    3769 non-null   int64  
 2   id_comercio_movimiento               3769 non-null   object 
 3   nombre_comercio_histo                3769 non-null   object 
 4   numero_cupon_movimiento              3769 non-null   int64  
 5   debito_credito_movimiento            3769 non-null   int64  
 6   producto_naranja_movimiento          3769 non-null   object 
 7   codigo_empresa_movimiento            3769 non-null   int64  
 8   tipo_producto_tarjeta_movimiento     3769 non-null   int64  
 9   plan_movimiento                      3769 non-null   int64  
 10  fecha_vto_cupon_movimiento           3769 non-null   object 
 11  fecha_presentacion_movimiento 

## 1. Curación

### Eliminar filas duplicadas

In [7]:
df = df[~df.duplicated()]

In [8]:
df['monto_compra_movimiento'].describe().round(2)

count      3769.00
mean       7505.14
std       16613.69
min      -35609.05
25%          -5.34
50%          40.33
75%        7903.68
max      162175.54
Name: monto_compra_movimiento, dtype: float64

### Ajuste por Inflación

In [9]:
df_infl = pd.read_csv('https://raw.githubusercontent.com/JIBarrionuevoGaltier/DiploDatos_2021_Mentoria_Grupo_2/main/notebooks_tp2/Indices%20de%20inflaci%C3%B3n.csv', 
                      parse_dates=['mes'])

In [10]:
df_infl

,mes,inflacion_mensual,inflacion_acumulada
0,2020-02-01,0.000,0.000
1,2020-03-01,0.033,0.033
2,2020-04-01,0.015,0.048
3,2020-05-01,0.015,0.063
4,2020-06-01,0.022,0.085
5,2020-07-01,0.019,0.104
6,2020-08-01,0.027,0.131
7,2020-09-01,0.028,0.159
8,2020-10-01,0.038,0.197
9,2020-11-01,0.032,0.229


In [11]:
# Procedimiento para ajustar la variable monto a la inflacion en el periodo muestreado

df['anio_mes_cupon'] = pd.DatetimeIndex(df['fecha_cupon_movimiento']).year * 100 + pd.DatetimeIndex(df['fecha_cupon_movimiento']).month
df_infl['anio_mes_infl'] = pd.DatetimeIndex(df_infl['mes']).year * 100 + pd.DatetimeIndex(df_infl['mes']).month

df = df.merge(df_infl[['anio_mes_infl', 'inflacion_acumulada']]
         ,left_on='anio_mes_cupon'
         ,right_on='anio_mes_infl'
         ,how='inner').drop(columns=['anio_mes_infl'])
df.loc[df.moneda_movimiento != 1,'monto_ajustado'] = \
                df[df.moneda_movimiento != 1]['monto_compra_movimiento'] / (1 + df['inflacion_acumulada'])

In [12]:
# Visualizamos montos ajustados en pesos luego del ajuste
df.loc[df['moneda_movimiento'] != 1, ['monto_compra_movimiento','monto_ajustado']].describe().round(2)

,monto_compra_movimiento,monto_ajustado
count,3749.00,3749.00
mean,7545.37,5056.97
std,16648.80,11152.22
min,-35609.05,-24256.85
25%,-5.34,-3.56
50%,42.35,28.85
75%,7932.87,5340.27
max,162175.54,109731.00


### Ajuste por Tipo de Cambio

In [13]:
# Obtención de la cotizacion del dolar para los meses en nuestro dataset
df_tdc = pd.read_csv('https://raw.githubusercontent.com/JIBarrionuevoGaltier/DiploDatos_2021_Mentoria_Grupo_2/main/notebooks_tp2/Tipos%20de%20cambio.csv',
                     parse_dates=['fecha_cotizacion'], dayfirst=True, delimiter=';')

df_tdc = df_tdc.sort_values(by=['fecha_cotizacion']).reset_index(level=None).drop(columns=['index'])
df_tdc['anio_mes'] = (df_tdc.fecha_cotizacion.dt.year * 100) + df_tdc.fecha_cotizacion.dt.month
df_tdc.head(10)

,fecha_cotizacion,compra,venta,anio_mes
0,2020-02-10,58.00,63.00,202002
1,2020-02-11,58.25,63.25,202002
2,2020-02-12,58.25,63.25,202002
3,2020-02-13,58.25,63.25,202002
4,2020-02-14,58.25,63.25,202002
5,2020-02-17,58.25,63.25,202002
6,2020-02-18,58.25,63.25,202002
7,2020-02-19,58.50,63.50,202002
8,2020-02-20,58.50,63.50,202002
9,2020-02-21,58.50,63.50,202002


In [14]:
# Procedimiento para calcular la fecha de cotizacion 
df_tdc['day'] = df_tdc.fecha_cotizacion.dt.day

meses_cdiez = df_tdc[df_tdc['day'].isin([10])]['anio_mes'].unique()
meses_sdiez = df_tdc[(~df_tdc['day'].isin([10])) & (~df_tdc['anio_mes'].isin(meses_cdiez))]['anio_mes'].unique()

ind_fechas = []
for i in meses_cdiez:
    ind_fechas.append(min((df_tdc[(df_tdc['anio_mes'] == i) & (df_tdc['day'] == 10)]['fecha_cotizacion'].index)))
for i in meses_sdiez:
    ind_fechas.append(min((df_tdc[(df_tdc['anio_mes'] == i) & (df_tdc['day'] > 10)]['fecha_cotizacion'].index)))

In [15]:
# Cotizacion del dia y el dia de vencimiento
df_tdc_cot = df_tdc.iloc[ind_fechas,:].sort_values(by=['fecha_cotizacion']).reset_index(level=None).drop(columns=['index'])
df_tdc_cot

,fecha_cotizacion,compra,venta,anio_mes,day
0,2020-02-10,58.00,63.00,202002,10
1,2020-03-10,59.00,64.00,202003,10
2,2020-04-13,61.75,66.75,202004,13
3,2020-05-11,64.25,69.25,202005,11
4,2020-06-10,66.50,71.50,202006,10
5,2020-07-13,69.50,74.50,202007,13
6,2020-08-10,72.00,77.00,202008,10
7,2020-09-10,74.00,79.00,202009,10
8,2020-10-13,76.25,82.25,202010,13
9,2020-11-10,78.75,84.75,202011,10


In [16]:
# Procesamos la variable anio_mes_cupon considerando la fecha de corte
df.fecha_cupon_movimiento = pd.to_datetime(df.fecha_cupon_movimiento)

def func_var(row):
    if row.fecha_cupon_movimiento.day > 24:
        if row.fecha_cupon_movimiento.month == 12:
            return 202101
        else:
            return row['anio_mes_cupon'] + 1
    else:
        return row['anio_mes_cupon']

df.loc[df.moneda_movimiento == 1,'anio_mes_cupon'] = df[df.moneda_movimiento == 1].apply(func_var, axis=1)

Mostramos un ejemplo donde se puede visualizar que anio_mes_cupon cambia para antes y despues de la fecha de corte

In [17]:
# Mergeamos con df original
df = df.merge(df_tdc_cot[['anio_mes', 'venta']]
                 ,left_on='anio_mes_cupon'
                 ,right_on='anio_mes'
                 ,how='inner').drop(columns=['anio_mes'])

# Aplicamos el tipo de cambio
df.loc[df.moneda_movimiento == 1,'monto_ajustado'] = \
                df[df.moneda_movimiento == 1]['monto_compra_movimiento'] * df['venta']

# Se descartan las columnas usadas para los calculos
df = df.drop(columns=['inflacion_acumulada','venta'])

# Redondeamos
df['monto_ajustado'] = df['monto_ajustado'].round(2)

In [18]:
# Visualizamos montos ajustados en dolares luego de aplicar el tipo de cambio
df.loc[df['moneda_movimiento'] == 1, ['monto_compra_movimiento','monto_ajustado']].describe().round(2)

,monto_compra_movimiento,monto_ajustado
count,20.00,20.00
mean,-36.24,-3642.43
std,76.30,7666.82
min,-184.75,-18475.00
25%,-18.94,-1912.94
50%,-0.69,-69.00
75%,8.00,800.00
max,20.00,2000.00


------

In [19]:
df['moneda_movimiento'].value_counts()

0    3749
1      20
Name: moneda_movimiento, dtype: int64

### 1.1 Asegurar IDs únicos

In [20]:
# Reagrupar los comercios que tienen mas de un nombre y concatenar todos sus nombres
df.nombre_comercio_histo = df.nombre_comercio_histo.str.strip()

df_unicos = (
    df[["id_comercio_movimiento", "nombre_comercio_histo"]].drop_duplicates()\
        .groupby("id_comercio_movimiento").agg({"nombre_comercio_histo": 'sum'})
).reset_index()

df_unicos = df_unicos.rename(columns={'nombre_comercio_histo':'nombre_comercio_concat'})

Reagrupamos los comercios que tienen mas de un nombre y concatenamos todos sus nombres en una nueva columna **nombre_comercio_concat**

In [21]:
# Unimos con dataframe original agregando la columna nueva con relacion 1:1 con el id
df = df.merge(df_unicos.drop_duplicates(subset=['id_comercio_movimiento'], keep='first'), on='id_comercio_movimiento', how='inner')

# Relacion 1:1 entre id_comercio_movimiento y la columna nueva con los nombres concatenados
df[['id_comercio_movimiento', 'nombre_comercio_concat']].drop_duplicates() \
.groupby(['id_comercio_movimiento']).count() # Deberia dar 3833 que es la cantidad de ids unicos

,nombre_comercio_concat
id_comercio_movimiento,
010099033,1
010099058,1
010099814,1
030090003,1
030090028,1
...,...
091099683,1
091099880,1
091099897,1


### 1.2. Asegurar el tipo de dato de codigo postal

In [22]:
df.domicilio_codigo_postal = df[df['domicilio_codigo_postal'].isna()!=True]['domicilio_codigo_postal'].apply(int).apply(str)
df.domicilio_codigo_postal = df.domicilio_codigo_postal.str.strip()

### 1.4. Reemplazar caracteres especiales

In [23]:
# Columnas que contienen strings
columnas_str = [
    'nombre_comercio_histo',
    'producto_naranja_movimiento',
    'nombre_local_histo',
    'estado_civil_descripcion',
    'sexo_descripcion',
    'pais',
    'provincia',
    'ciudad',
    'domicilio_barrio',
    'cargo_descripcion_histo',
    'nivel_estudio_descripcion_histo',
    'rel_vivienda_descripcion_histo',
    'nombre_comercio_concat'
]

In [24]:
# Expresion regular que incluye solo caracteres de a-z mayúscula o minúscula y 0-9
regex = r'[^a-zA-Z0-9 ]'

In [25]:
for col in columnas_str:
    df[col] = df[df[col].isnull()==False][col].apply(unidecode)

In [26]:
for col in columnas_str:
    df.loc[:, col] = df[df[col].isnull()==False][col].str.replace(regex, '')

<ipython-input-26-6865719e0677>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[df[col].isnull()==False][col].str.replace(regex, '')


### 1.5. Asegurar formato de fecha

In [27]:
# Columnas con fecha

col_fechas = [
    'fecha_vto_cupon_movimiento',
    'fecha_presentacion_movimiento',
    'fecha_cupon_movimiento',
    'fecha_carga_sistema_movimiento',
    'fecha_nacimiento',
    'fecha_extraccion_movimiento',
    'fecha_de_ingreso_histo'
]

In [28]:
df[col_fechas] = df[col_fechas].apply(pd.to_datetime)

In [29]:
df["fecha_cupon_movimiento"].max()

Timestamp('2021-07-10 00:00:00')

### 1.6. Agrupar valores de cargo_descripcion_histo

In [30]:
df = df[~(df['cargo_descripcion_histo'] == 'GERENTE')].copy()
df = df[~(df['cargo_descripcion_histo'] == 'JUEZ')].copy()

In [31]:
sec_salud = ['ENFERMERO','MEDICO']
sec_edu = ['DOCENTE','PROFESTECNICO']
sec_seg = ['EMPSEGPUBLICA','PORTEROCONSERJ','VIGILADOR']
sec_fin = ['PRESGARANTIA','PRESTCREDITO']
sec_Emp_Com = ['ADMINISTRATIVO','CAJERO','EMPDE COMERCIO','MAESTRANZA','JEFE','ENCARGADO','VENDEDORPROMOT', 'COCINERO','MOZO']
sec_ope = ['CHOFER','INDEPENDIENTE','OPERARIO','PEONEMBARCADOS','PROPIETARIO']
sec_no_ope = ['JUBILADO']
sec_sd = ['SIN DATOS']
#sec_pers_jer = ['GERENTE']
#sec_jud = ['JUEZ']

df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_salud)),'cargo_sector_desc_hist'] = 'Sector_Salud'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_edu)),'cargo_sector_desc_hist'] = 'Sector_Educativo'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_seg)),'cargo_sector_desc_hist'] = 'Sector_Seguridad'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_fin)),'cargo_sector_desc_hist'] = 'Sector_Financiero'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_Emp_Com)),'cargo_sector_desc_hist'] = 'Sector_Empleado_Comercio'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_ope)),'cargo_sector_desc_hist'] = 'Sector_Operativo'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_no_ope)),'cargo_sector_desc_hist'] = 'Sector_No_Operativo'
#df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_pers_jer)),'cargo_sector_desc_hist'] = 'Sector_Personal_Jerarquico'
#df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_jud)),'cargo_sector_desc_hist'] = 'Sector_Judicial'
df.loc[df['cargo_descripcion_histo'].str.contains('|'.join(sec_sd)),'cargo_sector_desc_hist'] = 'Sector_Sin_Datos'

In [32]:
df[df['cargo_sector_desc_hist'].isna()]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,domicilio_codigo_postal,domicilio_barrio,fecha_de_ingreso_histo,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist


In [33]:
df = df.reset_index().drop(columns=['index'])

-----

## 2 Data Enrichment

### 2.1. Cálculo de la edad de la persona

In [34]:
df['edad_cliente'] = ((df['fecha_cupon_movimiento'].max() - df['fecha_nacimiento'])/np.timedelta64(1, 'Y')).round(0)
df.edad_cliente = df.edad_cliente.astype('Int64', errors='ignore')

### 2.2 Cálculo de la antigüedad de la persona

In [35]:
df['antig_cliente']= ((df['fecha_cupon_movimiento'] - df['fecha_de_ingreso_histo'])/np.timedelta64(1, 'M'))
df['antig_cliente']= df.antig_cliente.apply(np.round).apply(int)
df[['fecha_cupon_movimiento', 'fecha_de_ingreso_histo', 'antig_cliente']]

,fecha_cupon_movimiento,fecha_de_ingreso_histo,antig_cliente
0,2021-05-28,2008-10-01,152
1,2021-05-28,2008-10-01,152
2,2021-05-10,2007-02-14,171
3,2021-05-10,2007-02-14,171
4,2021-06-03,2018-08-13,34
...,...,...,...
3755,2021-07-07,2017-12-19,43
3756,2021-07-05,2017-11-30,43
3757,2021-07-05,2017-11-30,43
3758,2021-07-04,2004-07-19,204


### 2.3 Reagrupamiento de **nombre_comercio_histo**

In [36]:
df.id_comercio_movimiento.astype(str)

0       030099091
1       030099091
2       030099168
3       030099168
4       030099168
          ...    
3755    030090700
3756    030091606
3757    030091606
3758    030096970
3759    030096970
Name: id_comercio_movimiento, Length: 3760, dtype: object

In [37]:
df.id_comercio_movimiento = df.id_comercio_movimiento.astype(str)

In [38]:
df['id_comercio_movimiento'] = df['id_comercio_movimiento'].str.zfill(9) # Se agrega el 0 a la izquierda

df['id_comercio_movimiento'].str.len().value_counts()

9    3760
Name: id_comercio_movimiento, dtype: int64

In [39]:
# Categorización
def func(row):
    return row.id_comercio_movimiento[0:1]

df['comercio_cat'] = df.apply(func, axis=1)

In [40]:
# Visualizar la columna de comercios recategorizada, segun cuantos id_comercios y cuantas transacciones se abarcan.
df_comercios = df['comercio_cat'].value_counts().to_frame().reset_index().rename(
                                    {'index':'comercio_cat',
                                     'comercio_cat':'frecuencia_ts'}, axis=1)

df_com_freq = pd.DataFrame(df.groupby(['comercio_cat'])['id_comercio_movimiento'].nunique().sort_values(ascending=True)) \
                            .reset_index().rename({
                                'id_comercio_movimiento':'frecuencia_idcom'
                            }, axis=1)
df_com_freq = df_com_freq.merge(df_comercios, on='comercio_cat', how='inner')
df_com_freq

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_com_freq.sort_values(by='frecuencia_idcom'))

  comercio_cat  frecuencia_idcom  frecuencia_ts
0            0               249           3760


### 3.1 Análisis e imputación de Datos Nulos

Se puede observar cierta sistematicidad en la pérdida de valores relacionados a datos personales de los clientes. A continuación procedemos a analizar cada columna con valores faltantes.

#### Fecha carga sistema movimiento

In [41]:
df.drop('fecha_carga_sistema_movimiento', axis=1, inplace=True)

#### Tipo_prestamo_movimiento, Nombre_local_histo y Fecha_extraccion_movimiento 

In [42]:
df.drop(columns = ['tipo_prestamo_movimiento','nombre_local_histo','fecha_extraccion_movimiento'], inplace=True)

#### Datos Faltantes Personales de los clientes

In [43]:
df[df['sexo_descripcion'] == '']

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
238,ad0525299ff5ef0a4ff16b5012f855e3938,0,040111140,PES LINK TN,560548,1,PL,1,0,1,...,SIN DATOS,PRIMARIOS,Propia,202106,-0.40,PES LINK TN,Sector_Sin_Datos,73,65,0
247,ad0525299ff5ef0a4ff16b5012f855e3938,0,040111140,PES LINK TN,560548,1,PL,1,0,1,...,SIN DATOS,PRIMARIOS,Propia,202106,-0.40,PES LINK TN,Sector_Sin_Datos,73,65,0
530,2b1fc89ed6de585bb7a2133c21182a26174,0,064101401,NARANJA PLUS PAGO FACIL,741481,0,PC,1,3,9,...,SIN DATOS,SECUNDARIOS,Otros,202105,10391.77,NARANJA PLUS PAGO FACIL,Sector_Sin_Datos,61,51,0
591,2b1fc89ed6de585bb7a2133c21182a26174,0,064101401,NARANJA PLUS PAGO FACIL,741481,0,PC,1,3,9,...,SIN DATOS,SECUNDARIOS,Otros,202105,10391.77,NARANJA PLUS PAGO FACIL,Sector_Sin_Datos,61,51,0


In [44]:
columna = ['sexo_descripcion']
df['sexo_descripcion'] = df['sexo_descripcion'].replace('', 'Sin Datos')
df['sexo_descripcion'] = df['sexo_descripcion'].replace(np.nan, 'Sin Datos')
#df['sexo_descripcion']= df.sexo_descripcion.fillna(value=np.nan)
#const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
#df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [45]:
#df['sexo_descripcion'] = df['sexo_descripcion'].astype(str)
df['sexo_descripcion'].value_counts(dropna=False)

Mujer        1935
Hombre       1810
Sin Datos      15
Name: sexo_descripcion, dtype: int64

##### Datos faltantes Nivel de Estudio

In [46]:
df.nivel_estudio_descripcion_histo.value_counts()

SECUNDARIOS       2154
PRIMARIOS          935
TERCIARIOS         384
UNIVERSITARIOS     203
Name: nivel_estudio_descripcion_histo, dtype: int64

In [47]:
columna = ['nivel_estudio_descripcion_histo']
df_faltante_estud = pd.DataFrame(df[df['nivel_estudio_descripcion_histo'].isna()])
df_faltante_estud[columna].isna().sum()

nivel_estudio_descripcion_histo    84
dtype: int64

In [48]:
dnis_faltantes = df_faltante_estud.dni_titular_movimiento.values
df[(df['dni_titular_movimiento'].isin(dnis_faltantes)) & (~df['nivel_estudio_descripcion_histo'].isnull())]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat


Observamos que la pérdida de datos de nivel de estudio también es sistemática, no pudiendose recuperar este dato, procedemos a imputar dichas filas con Sin datos. 

In [49]:
columna = ['nivel_estudio_descripcion_histo']
#df['sexo_descripcion']= df.sexo_descripcion.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [50]:
df.nivel_estudio_descripcion_histo.value_counts()

SECUNDARIOS       2154
PRIMARIOS          935
TERCIARIOS         384
UNIVERSITARIOS     203
Sin Datos           84
Name: nivel_estudio_descripcion_histo, dtype: int64

##### Datos faltantes Vivienda

In [51]:
df.rel_vivienda_descripcion_histo.value_counts()

Propia         2688
De familiar     304
Otros           183
Alquilada       177
Name: rel_vivienda_descripcion_histo, dtype: int64

In [52]:
columna = ['rel_vivienda_descripcion_histo']
df_faltante_vivi = pd.DataFrame(df[df['rel_vivienda_descripcion_histo'].isna()])
df_faltante_vivi[columna].isna().sum()

rel_vivienda_descripcion_histo    408
dtype: int64

In [53]:
dnis_faltantes = df_faltante_vivi.dni_titular_movimiento.values
df[(df['dni_titular_movimiento'].isin(dnis_faltantes)) & (~df['rel_vivienda_descripcion_histo'].isnull())]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat


In [54]:
columna = ['rel_vivienda_descripcion_histo']
#df["nivel_estudio_descripcion_histo"]= df.nivel_estudio_descripcion_histo.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [55]:
df.rel_vivienda_descripcion_histo.value_counts()

Propia         2688
Sin Datos       408
De familiar     304
Otros           183
Alquilada       177
Name: rel_vivienda_descripcion_histo, dtype: int64

##### Datos faltantes Edad

Procedemos a imputar usando IterativeImputer (en su operación por defecto BayesianRigde) la variable edad_cliente. Para imputar usamos las columnas **edad_cliente** y **antig_cliente**.

In [56]:
df.shape

(3760, 44)

In [57]:
df[df['edad_cliente'].isna()]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
830,540bbeccd826ea8274a77b3382617609868,0,064101401,NARANJA PLUS PAGO FACIL,525326,0,PC,1,3,6,...,OPERARIO,PRIMARIOS,Propia,202107,25234.54,NARANJA PLUS PAGO FACIL,Sector_Operativo,<NA>,262,0
862,540bbeccd826ea8274a77b3382617609868,0,064101401,NARANJA PLUS PAGO FACIL,525326,0,PC,1,3,6,...,OPERARIO,PRIMARIOS,Propia,202107,25234.54,NARANJA PLUS PAGO FACIL,Sector_Operativo,<NA>,262,0
1598,5889a3db2e32aafde9be555610f7ece9395,0,078197180,TASI 1 SAN JUAN 1,288771,1,PL,1,0,1,...,SIN DATOS,SECUNDARIOS,Propia,202105,-51.95,TASI 1 SAN JUAN 1,Sector_Sin_Datos,<NA>,125,0
2085,65e0b17d1f01a21604e9c35b924db37b466,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,JUBILADO,PRIMARIOS,Propia,202106,-1277.91,TARJETA NARANJA SUCRE,Sector_No_Operativo,<NA>,210,0
2105,45e319417984886120ca2d3e1b008273336,0,010099058,TARJETA NARANJA SUCRE,630328,0,PL,1,3,3,...,SIN DATOS,PRIMARIOS,Propia,202106,1152.73,TARJETA NARANJA SUCRE,Sector_Sin_Datos,<NA>,24,0
2420,65e0b17d1f01a21604e9c35b924db37b466,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,JUBILADO,PRIMARIOS,Propia,202106,-1277.91,TARJETA NARANJA SUCRE,Sector_No_Operativo,<NA>,210,0
2440,45e319417984886120ca2d3e1b008273336,0,010099058,TARJETA NARANJA SUCRE,630328,0,PL,1,3,3,...,SIN DATOS,PRIMARIOS,Propia,202106,1152.73,TARJETA NARANJA SUCRE,Sector_Sin_Datos,<NA>,24,0
2732,65e0b17d1f01a21604e9c35b924db37b466,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,JUBILADO,PRIMARIOS,Propia,202107,-1275.36,TARJETA NARANJA SUCRE,Sector_No_Operativo,<NA>,211,0
3001,65e0b17d1f01a21604e9c35b924db37b466,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,JUBILADO,PRIMARIOS,Propia,202107,-1275.36,TARJETA NARANJA SUCRE,Sector_No_Operativo,<NA>,211,0
3176,1f67ece7d8ec69dcde4b3bb0fe85f5b8422,0,030090504,TARJETA NARANJA MAR DEL PLATA,374023,0,PL,1,3,1,...,SIN DATOS,TERCIARIOS,Propia,202105,4.17,TARJETA NARANJA MAR DEL PLATA,Sector_Sin_Datos,<NA>,31,0


In [58]:
#df['edad_cliente'] = df['edad_cliente'].replace(np.nan, 0)

In [59]:
#df[df['edad_cliente'].isna()]

In [60]:
#, missing_values=0
imp = IterativeImputer(max_iter=10, random_state=0)
imp_columns = ['edad_cliente', 'antig_cliente']
x_train = df[imp_columns]
imp.fit(x_train)

x_test = df[imp_columns]
x_test = np.round(imp.transform(x_test))
x_test = pd.DataFrame(x_test, columns=imp_columns)
x_test

,edad_cliente,antig_cliente
0,78.0,152.0
1,78.0,152.0
2,35.0,171.0
3,35.0,171.0
4,43.0,34.0
...,...,...
3755,61.0,43.0
3756,49.0,43.0
3757,49.0,43.0
3758,51.0,204.0


In [61]:
df['edad_cliente'] = x_test['edad_cliente']

In [62]:
df['edad_cliente'].describe().round(2)

count    3760.00
mean       49.74
std        14.83
min        23.00
25%        38.00
50%        48.00
75%        60.00
max       121.00
Name: edad_cliente, dtype: float64

In [63]:
df[df['edad_cliente'].isna()]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat


La columna fecha_nacimiento se deja sin imputar debido a que su unica finalidad era obtener la edad del cliente. Por lo tanto, no se tiene en cuenta.

In [64]:
df.drop('fecha_nacimiento', inplace=True, axis=1)

#### Datos Faltantes Geograficos de los clientes

##### Sistematicidad en la pérdida

In [65]:
col_geo = ['domicilio_codigo_postal', 'pais', 'provincia', 'ciudad', 'domicilio_barrio']
df[df['domicilio_codigo_postal'].isna()][col_geo].isna().sum()

domicilio_codigo_postal    27
pais                       27
provincia                  27
ciudad                     27
domicilio_barrio           27
dtype: int64

Tenemos un total de 24 provincias, el resto son registros NaN.

In [66]:
df['domicilio_codigo_postal'] = df['domicilio_codigo_postal'].replace('0',np.nan)
df[df['domicilio_codigo_postal']=='0']

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat


In [67]:
df['ciudad'] = df['ciudad'].str.strip()
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 0


,pais,provincia,ciudad,domicilio_codigo_postal


In [68]:
df.loc[df['ciudad'].isin(['CAMPO HERRERA']),'domicilio_codigo_postal'] = '4105'
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 0


,pais,provincia,ciudad,domicilio_codigo_postal


In [69]:
df[df['pais'].isnull()][['pais','provincia','ciudad','domicilio_codigo_postal','domicilio_barrio']].isna().sum()

pais                       106
provincia                  106
ciudad                     106
domicilio_codigo_postal     27
domicilio_barrio            27
dtype: int64

In [70]:
cp_ciu_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['ciudad'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_ciu_nan)][['domicilio_codigo_postal','ciudad']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad
1239,1055,CIUDAD AUTONOMA DE BUENOS AIRES
3352,1055,NaN
1963,1754,NaN
1344,1754,SAN JUSTO
352,1832,LOMAS DE ZAMORA
...,...,...
2051,8300,NaN
84,8300,NEUQUEN
364,8332,GENERAL ROCA
2267,8332,NaN


In [71]:
df.loc[:,'ciudad'] = df['ciudad'].str.upper()
df_dp_cd = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==False)]\
            [['domicilio_codigo_postal','ciudad']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_cd = df_dp_cd.rename(columns={'ciudad': 'ciudad_0', 'domicilio_codigo_postal': 'cp'})
df_dp_cd

,cp,ciudad_0
1239,1055,CIUDAD AUTONOMA DE BUENOS AIRES
1344,1754,SAN JUSTO
352,1832,LOMAS DE ZAMORA
2021,1884,BERAZATEGUI
1365,2000,ROSARIO SANTA FE
286,2000,ROSARIO
2,2300,RAFAELA
30,2421,MORTEROS
2002,2804,CAMPANA
1072,3000,SANTA FE


Realizamos merge sobre el dataframe creado, e imputamos los datos de ciudad.

In [72]:
df = df.merge(df_dp_cd.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,ciudad_0
0,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,SECUNDARIOS,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,NaN
1,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,SECUNDARIOS,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,NaN
2,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,PRIMARIOS,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,RAFAELA
3,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,PRIMARIOS,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,RAFAELA
4,daebbfeda20de1dd3167cfc1dd4aa33e339,0,030099168,TARJETA NARANJA RAFAELA,275812,0,PL,1,3,1,...,SECUNDARIOS,Propia,202106,62.93,TARJETA NARANJA RAFAELA,Sector_Sin_Datos,43.0,34,0,RAFAELA


In [73]:
# Sustituimos los valores nulos de la columna provincia (provincia_x) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['ciudad_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==True)]\
    [['domicilio_codigo_postal','ciudad','ciudad_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad,ciudad_0
2004,1980,NaN,NaN
100,2064,NaN,NaN
2204,4115,NaN,NaN
2247,7240,NaN,NaN
2050,8370,NaN,NaN


In [74]:
df = df.drop(columns=['ciudad_0'])

Procedemos a imputar **provincia** de la misma manera que **ciudad**.

In [75]:
cp_prov_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['provincia'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_prov_nan)][['domicilio_codigo_postal','provincia']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia
3352,1055,NaN
1239,1055,CAPITAL FEDERAL
1963,1754,NaN
1344,1754,BUENOS AIRES
352,1832,BUENOS AIRES
453,1832,NaN
2270,1884,NaN
2021,1884,BUENOS AIRES
286,2000,SANTA FE
3231,2000,NaN


In [76]:
df.loc[:,'provincia'] = df['provincia'].str.upper()
df_dp_pv = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==False)]\
            [['domicilio_codigo_postal','provincia']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_pv = df_dp_pv.rename(columns={'provincia': 'provincia_0', 'domicilio_codigo_postal': 'cp'})
df_dp_pv

,cp,provincia_0
1239,1055,CAPITAL FEDERAL
1344,1754,BUENOS AIRES
352,1832,BUENOS AIRES
2021,1884,BUENOS AIRES
2004,1980,BUENOS AIRES
286,2000,SANTA FE
100,2064,BUENOS AIRES
2,2300,SANTA FE
30,2421,CORDOBA
2002,2804,BUENOS AIRES


In [77]:
df = df.merge(df_dp_pv.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,provincia_0
0,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,SECUNDARIOS,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,NaN
1,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,SECUNDARIOS,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,NaN
2,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,PRIMARIOS,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,SANTA FE
3,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,PRIMARIOS,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,SANTA FE
4,daebbfeda20de1dd3167cfc1dd4aa33e339,0,030099168,TARJETA NARANJA RAFAELA,275812,0,PL,1,3,1,...,SECUNDARIOS,Propia,202106,62.93,TARJETA NARANJA RAFAELA,Sector_Sin_Datos,43.0,34,0,SANTA FE


In [78]:
# Sustituimos los valores nulos de la columna provincia (provincia_0) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.

df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==True)]\
    [['domicilio_codigo_postal','provincia','provincia_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia,provincia_0
2247,7240,NaN,NaN
2050,8370,NaN,NaN


In [79]:
df = df.drop(columns=['provincia_0'])

In [80]:
df[col_geo].isna().sum()

domicilio_codigo_postal     31
pais                       106
provincia                   35
ciudad                      49
domicilio_barrio            27
dtype: int64

##### Imputación con dataset externo

In [81]:
df_cp = pd.read_csv('https://raw.githubusercontent.com/JIBarrionuevoGaltier/localidades_AR/master/localidades_cp_maestro.csv')
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144.0,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145.0,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146.0,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147.0,2


In [82]:
# Realizamos una curación de datos sobre este dataset.

df_cp.cp = df_cp[df_cp['cp'].isna()!=True]['cp'].apply(int).apply(str)
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147,2


Observamos si todos los Códigos Postales de nuestro DataFrame original se encuentran en los objetos del merge.

In [83]:
df_cp = df_cp[~df_cp['cp'].isnull()]
df_cp.isna().sum()

provincia       0
id              0
localidad       0
cp              0
id_prov_mstr    0
dtype: int64

In [84]:
df_cp = df_cp.rename(columns={'provincia': 'provincia_0'})
df_cp.head()

,provincia_0,id,localidad,cp,id_prov_mstr
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147,2
5,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1148,2


In [85]:
df.shape

(3760, 43)

Procedemos a hacer el merge de los datasets, a través de la columna cp (Codigo Postal)

In [86]:
# Unimos por cp, eliminamos duplicados, conservamos solo cps del dataframe original
df = df.merge(df_cp[['provincia_0','localidad','cp']].drop_duplicates(subset=['cp'], keep='first'), 
                           how='left',
                           left_on = 'domicilio_codigo_postal', 
                           right_on = 'cp')\
                    .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,provincia_0,localidad
0,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,Salta,SACHA PERA
1,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,Salta,SACHA PERA
2,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,Santa Fe,PUEBLO TERRAGNI
3,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,Santa Fe,PUEBLO TERRAGNI
4,daebbfeda20de1dd3167cfc1dd4aa33e339,0,030099168,TARJETA NARANJA RAFAELA,275812,0,PL,1,3,1,...,Propia,202106,62.93,TARJETA NARANJA RAFAELA,Sector_Sin_Datos,43.0,34,0,Santa Fe,PUEBLO TERRAGNI


In [87]:
df.shape

(3760, 45)

In [88]:
df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['localidad'].str.upper()

In [89]:
df = df.drop(columns=['provincia_0','localidad'])

In [90]:
df[col_geo].isna().sum()

domicilio_codigo_postal     31
pais                       106
provincia                   27
ciudad                      33
domicilio_barrio            27
dtype: int64

In [91]:
df.shape

(3760, 43)

In [92]:
df.drop(columns=['pais','domicilio_barrio'], inplace=True)

In [93]:
columna = ['domicilio_codigo_postal', 'provincia', 'ciudad']
#df['sexo_descripcion']= df.sexo_descripcion.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [94]:
df[['domicilio_codigo_postal', 'provincia', 'ciudad']].isna().sum()

domicilio_codigo_postal    0
provincia                  0
ciudad                     0
dtype: int64

In [95]:
df.shape

(3760, 41)

#### Nueva eliminación de filas duplicadas 

In [96]:
df = df[~df.duplicated()]

#### Variables categoricas (Reagrupación)

In [97]:
df.shape

(1915, 41)

##### Estado Civil

In [98]:
df.estado_civil_descripcion.value_counts()

Solteroa               991
Casadoa                704
Divorciadoa             77
Viudoa                  69
Concubinoa              42
Separacion de hecho     23
Sin Datos                7
Novioa                   2
Name: estado_civil_descripcion, dtype: int64

Observamos que existe una alta frecuencia en las categorías Solteros y Casados, perdiendo representativadad en el resto de los estados civiles, por lo cual decidimos agrupar a la mismas en una misma categoría.

In [99]:
soltero = ['Solteroa']
casado = ['Casadoa']
otros = ['Divorciadoa','Viudoa','Concubinoa','Separacion de hecho','Novioa']
sin_datos = ['Sin Datos']

df.loc[df['estado_civil_descripcion'].str.contains('|'.join(soltero)),'estado_civil_cat'] = 'Soltero'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(casado)),'estado_civil_cat'] = 'Casado'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(otros)),'estado_civil_cat'] = 'Otros'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(sin_datos)),'estado_civil_cat'] = 'Sin_datos'

In [100]:
df.estado_civil_cat.value_counts()

Soltero      991
Casado       704
Otros        213
Sin_datos      7
Name: estado_civil_cat, dtype: int64

Viendo las frecuencias por provincias, vamos a recategorizar sobre una nueva columna, las provincias por regiones.

In [101]:
df[df['provincia'] == '']

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat
1987,d0dfca39178b2e72d46712aa9ff3cb5f031,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,PRIMARIOS,Otros,202106,-3.56,TARJETA NARANJA SUCRE,Sector_Sin_Datos,39.0,175,0,Soltero
2652,d0dfca39178b2e72d46712aa9ff3cb5f031,0,010099058,TARJETA NARANJA SUCRE,1,1,PL,0,0,1,...,PRIMARIOS,Otros,202107,-3.55,TARJETA NARANJA SUCRE,Sector_Sin_Datos,39.0,176,0,Soltero


In [102]:
dic_region = {'REGION_NOROESTE': ['JUJUY','SALTA','TUCUMAN','CATAMARCA','SGO DEL ESTERO'],
              'REGION_NORDESTE': ['CHACO','FORMOSA','CORRIENTES','MISIONES'], 
              'REGION_PAMPEANA': ['CORDOBA','BUENOS AIRES','CAPITAL FEDERAL','ENTRE RIOS','LA PAMPA','SANTA FE'], 
              'REGION_CUYO': ['SAN JUAN','SAN LUIS','LA RIOJA','MENDOZA'], 
              'REGION_PATAGONIA': ['SANTA CRUZ','TIERRA DEL FUEGO','RIO NEGRO','NEUQUEN','CHUBUT'],
              'SIN_DATOS': ['Sin Datos', '']}

df['region']= df['provincia']
for i in dic_region:
    df['region'] = df['region'].replace(dic_region[i], i)
    

df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat,region
0,50db35e67141a73d097d268ebf71d7fd331,0,030099091,TARJETA NARANJA METAN,953940,0,PL,1,3,1,...,Otros,202105,5.84,TARJETA NARANJA METAN,Sector_Sin_Datos,78.0,152,0,Casado,REGION_NOROESTE
2,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,274882,0,PL,1,3,1,...,Propia,202105,25.30,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,171,0,Soltero,REGION_PAMPEANA
4,daebbfeda20de1dd3167cfc1dd4aa33e339,0,030099168,TARJETA NARANJA RAFAELA,275812,0,PL,1,3,1,...,Propia,202106,62.93,TARJETA NARANJA RAFAELA,Sector_Sin_Datos,43.0,34,0,Soltero,REGION_PAMPEANA
5,1eabd3ed6213cf09d37fcfcb212b084f086,0,030099168,TARJETA NARANJA RAFAELA,276289,0,PL,1,3,1,...,Propia,202106,2.33,TARJETA NARANJA RAFAELA,Sector_Empleado_Comercio,29.0,89,0,Soltero,REGION_PAMPEANA
6,a2fa1760229b4238eb718db5bdedd57e237,0,030099168,TARJETA NARANJA RAFAELA,276318,0,PL,1,3,1,...,Propia,202106,32.59,TARJETA NARANJA RAFAELA,Sector_Operativo,35.0,172,0,Soltero,REGION_PAMPEANA


In [103]:
df['region'].value_counts(dropna=False)

REGION_PAMPEANA     991
REGION_NOROESTE     353
REGION_NORDESTE     213
REGION_CUYO         193
REGION_PATAGONIA    149
SIN_DATOS            16
Name: region, dtype: int64

In [104]:
df[df['region'].isna()]

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat,region


In [105]:
df_prod = df['producto_naranja_movimiento'].value_counts().to_frame().reset_index().rename(
                                    {'index':'producto_naranja_movimiento',
                                     'producto_naranja_movimiento':'frecuencia_pr'}, axis=1)

df_prod_dni = pd.DataFrame(df.groupby(['producto_naranja_movimiento'])['dni_titular_movimiento'].nunique() \
                            .sort_values(ascending=True)) \
                            .reset_index()

df_prod_dni = df_prod_dni.merge(df_prod, on='producto_naranja_movimiento', how='inner')
df_prod_dni

,producto_naranja_movimiento,dni_titular_movimiento,frecuencia_pr
0,PP,17,26
1,PN,43,49
2,PC,339,665
3,PL,668,1175


Observamos que la distribución que resulta de esta transformacion no se parece a una normal. Por lo tanto, concluimos que se pierde mucha información y las ganancias obtenidas son bajas. Decidimos entonces, conservar la distribución original de esta variable.

### 3.4.1 Seleccion de columnas a utilizar

In [106]:
df_final = df[['dni_titular_movimiento', 'fecha_cupon_movimiento', 'moneda_movimiento', 'producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento',
               'anio_mes_cupon', 'sexo_descripcion', 'monto_ajustado',
               'cargo_sector_desc_hist', 'edad_cliente', 'antig_cliente', 'comercio_cat', 
               'estado_civil_cat', 'region']]

In [107]:
df_final.shape

(1915, 14)

In [108]:
df_final

,dni_titular_movimiento,fecha_cupon_movimiento,moneda_movimiento,producto_naranja_movimiento,tipo_producto_tarjeta_movimiento,anio_mes_cupon,sexo_descripcion,monto_ajustado,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat,region
0,50db35e67141a73d097d268ebf71d7fd331,2021-05-28,0,PL,3,202105,Mujer,5.84,Sector_Sin_Datos,78.0,152,0,Casado,REGION_NOROESTE
2,a2fa1760229b4238eb718db5bdedd57e237,2021-05-10,0,PL,3,202105,Hombre,25.30,Sector_Operativo,35.0,171,0,Soltero,REGION_PAMPEANA
4,daebbfeda20de1dd3167cfc1dd4aa33e339,2021-06-03,0,PL,3,202106,Mujer,62.93,Sector_Sin_Datos,43.0,34,0,Soltero,REGION_PAMPEANA
5,1eabd3ed6213cf09d37fcfcb212b084f086,2021-06-08,0,PL,3,202106,Hombre,2.33,Sector_Empleado_Comercio,29.0,89,0,Soltero,REGION_PAMPEANA
6,a2fa1760229b4238eb718db5bdedd57e237,2021-06-08,0,PL,3,202106,Hombre,32.59,Sector_Operativo,35.0,172,0,Soltero,REGION_PAMPEANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,5d2d5bcc1c58727b1eb26426cac675b1422,2021-07-08,0,PL,3,202107,Mujer,1.70,Sector_Sin_Datos,70.0,71,0,Casado,REGION_PAMPEANA
3752,fb111ce38f13582c6e770c1950b6e36b910,2021-07-08,0,PL,0,202107,Hombre,0.99,Sector_Sin_Datos,50.0,159,0,Soltero,REGION_PAMPEANA
3754,f28060b0f1ed7c40190ce903d11a039f230,2021-07-07,0,PL,3,202107,Hombre,3.80,Sector_Sin_Datos,61.0,43,0,Casado,REGION_PAMPEANA
3756,b15f561adc3fc82655f33f2e6b2ed5a5532,2021-07-05,0,PL,3,202107,Hombre,26.30,Sector_Sin_Datos,49.0,43,0,Soltero,REGION_PAMPEANA


In [109]:
df_final['moneda_movimiento'].value_counts()

0    1905
1      10
Name: moneda_movimiento, dtype: int64

In [110]:
df_final.isna().sum()

dni_titular_movimiento              0
fecha_cupon_movimiento              0
moneda_movimiento                   0
producto_naranja_movimiento         0
tipo_producto_tarjeta_movimiento    0
anio_mes_cupon                      0
sexo_descripcion                    0
monto_ajustado                      0
cargo_sector_desc_hist              0
edad_cliente                        0
antig_cliente                       0
comercio_cat                        0
estado_civil_cat                    0
region                              0
dtype: int64

In [111]:
df_final.shape

(1915, 14)

## Consideraciones en el tratamiento de las variables

### Codificación

In [112]:
ordinal_ft = 'dni_titular_movimiento'
target = 'monto_ajustado'
#date_num = 'anio_mes_cupon'
num_features = ['anio_mes_cupon', 'edad_cliente', 'antig_cliente', 'fecha_cupon_movimiento']
trans_ft = ['producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento', 'moneda_movimiento', 'comercio_cat']
client_ft = ['sexo_descripcion', 'cargo_sector_desc_hist', 'estado_civil_cat', 'region']

In [113]:
# Cambiamos los valores para moneda y 

dic_tipo_prod = {42: 'American_express', 44: 'American_express_gold', 0: 'Clasica', 32: 'Master_internacional',
    31: 'Master_nacional', 3: 'Naranja', 4: 'Naranja_virtual', 24: 'Naranja_visa_mini_int', 23: 'Naranja_visa_mini_nac',
    12: 'Naranja_visa_internac.', 11: 'Naranja_visa_nacional', -1: 'Sin_Datos', 22: 'Visa_internacional', 21: 'Visa_nacional'}
dic_monedas = {0: 'Pesos',1: 'Dolares',3: 'Zeta'}
df_final.tipo_producto_tarjeta_movimiento = df_final.tipo_producto_tarjeta_movimiento.replace(dic_tipo_prod)
df_final.moneda_movimiento = df_final.moneda_movimiento.replace(dic_monedas)

C:\Users\anto_\.conda\envs\diplodatos-ayvd\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [114]:
df_final.moneda_movimiento.value_counts()

Pesos      1905
Dolares      10
Name: moneda_movimiento, dtype: int64

In [115]:
# Ordenamos por fecha
df = df_final.sort_values(by = ['fecha_cupon_movimiento'], ascending = True).copy()
# Transformacion de fecha a numerica
df['fecha_cupon_movimiento'] = df['fecha_cupon_movimiento'].values.astype(float)/10**11
# Transformacion de tipo de moneda a string
df['moneda_movimiento'] = df['moneda_movimiento'].astype(str)

# Codificación
cat_transformer = OneHotEncoder(handle_unknown='ignore')

encoder = ColumnTransformer(
    transformers=[
        ('dni', 'drop', [ordinal_ft]),
        ('num', 'passthrough', num_features),
   #     ('date_num', discretizer, [date_num]),
        ('trans', cat_transformer, trans_ft),
        ('client', cat_transformer, client_ft),
        ('target', 'passthrough', [target])])
df_enc = encoder.fit_transform(df)
cols = encoder.get_feature_names()

C:\Users\anto_\.conda\envs\diplodatos-ayvd\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [116]:
df['moneda_movimiento'].value_counts()

Pesos      1905
Dolares      10
Name: moneda_movimiento, dtype: int64

In [117]:
cols_o = np.hstack([[ordinal_ft],cols])
df_stack = np.hstack([df[[ordinal_ft]],df_enc])
df_encode = pd.DataFrame(df_stack , columns=cols_o)

### Agrupación por Mes

In [118]:
# Renombramos
df_encode.rename({'edad_cliente':'client__edad',
                         'antig_cliente':'client__antig'}, axis=1, inplace=True)
cols = df_encode.columns

# Funciones de agregacion para cada columna
aggr = {} 
aggr.update(dict.fromkeys([x for x in cols if 'client__' in x], 'max')) # Incluye a edad y antiguedad
aggr.update(dict.fromkeys([x for x in cols if 'trans__' in x], 'sum'))
aggr.update({target:'sum'})

In [119]:
num_features = ['anio_mes_cupon', 'client__edad', 'client__antig', 'fecha_cupon_movimiento']

# Convertimos las columnas categoricas de la transaccion a numericas para poder sumarizarlas
df_encode[num_features + [x for x in cols if 'trans' in x]] = \
df_encode[num_features + [x for x in cols if 'trans' in x]].apply(pd.to_numeric)

# Agrupamiento
group = ['dni_titular_movimiento', 'anio_mes_cupon']

df_mes = df_encode.groupby(group).agg(aggr).reset_index() # edad y antiguedad

In [120]:
df_mes.shape

(1531, 37)

In [121]:
df_mes

,dni_titular_movimiento,anio_mes_cupon,client__edad,client__antig,client__x0_Hombre,client__x0_Mujer,client__x0_Sin Datos,client__x1_Sector_Educativo,client__x1_Sector_Empleado_Comercio,client__x1_Sector_Financiero,...,trans__x0_PN,trans__x0_PP,trans__x1_Clasica,trans__x1_Master_internacional,trans__x1_Naranja,trans__x1_Visa_internacional,trans__x2_Dolares,trans__x2_Pesos,trans__x3_0,monto_ajustado
0,004aadd4e9f2aa2249719777be7188fb625,202105.0,34.0,72.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1177.85
1,004b7bb2c92a5dfa872165ee95e4ebd8937,202105.0,33.0,55.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.91
2,004b7bb2c92a5dfa872165ee95e4ebd8937,202106.0,33.0,55.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.78
3,008d748c1f0d5e7f654e5f49d9a09498693,202105.0,37.0,85.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,184.63
4,0108e33f28f7efdfb8798f80bc20426d757,202106.0,34.0,45.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-41.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,fda5249ba280b05705c4d318d0f873b7869,202107.0,72.0,116.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,31120.95
1527,fdf06f117bbcb9a87b2c2a24ea63f39a552,202106.0,32.0,69.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,10.01
1528,fe71528b4da02bf9a68588a41c7c7704743,202106.0,48.0,270.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.99
1529,fe83942b407a1f868cb48b2bbeab4988255,202105.0,79.0,119.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,643.74


### Guardado

Decidimos usar el formato 'parquet' para guardar el dataset, por ser mas liviano y rapido que csv

In [122]:
df_mes.to_parquet('df_supervisado_agrupado_test.parquet')

In [123]:
df_mes.columns

Index(['dni_titular_movimiento', 'anio_mes_cupon', 'client__edad',
       'client__antig', 'client__x0_Hombre', 'client__x0_Mujer',
       'client__x0_Sin Datos', 'client__x1_Sector_Educativo',
       'client__x1_Sector_Empleado_Comercio', 'client__x1_Sector_Financiero',
       'client__x1_Sector_No_Operativo', 'client__x1_Sector_Operativo',
       'client__x1_Sector_Salud', 'client__x1_Sector_Seguridad',
       'client__x1_Sector_Sin_Datos', 'client__x2_Casado', 'client__x2_Otros',
       'client__x2_Sin_datos', 'client__x2_Soltero', 'client__x3_REGION_CUYO',
       'client__x3_REGION_NORDESTE', 'client__x3_REGION_NOROESTE',
       'client__x3_REGION_PAMPEANA', 'client__x3_REGION_PATAGONIA',
       'client__x3_SIN_DATOS', 'trans__x0_PC', 'trans__x0_PL', 'trans__x0_PN',
       'trans__x0_PP', 'trans__x1_Clasica', 'trans__x1_Master_internacional',
       'trans__x1_Naranja', 'trans__x1_Visa_internacional',
       'trans__x2_Dolares', 'trans__x2_Pesos', 'trans__x3_0',
       'monto_ajust

In [124]:
df_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dni_titular_movimiento               1531 non-null   object 
 1   anio_mes_cupon                       1531 non-null   float64
 2   client__edad                         1531 non-null   float64
 3   client__antig                        1531 non-null   float64
 4   client__x0_Hombre                    1531 non-null   float64
 5   client__x0_Mujer                     1531 non-null   float64
 6   client__x0_Sin Datos                 1531 non-null   float64
 7   client__x1_Sector_Educativo          1531 non-null   float64
 8   client__x1_Sector_Empleado_Comercio  1531 non-null   float64
 9   client__x1_Sector_Financiero         1531 non-null   float64
 10  client__x1_Sector_No_Operativo       1531 non-null   float64
 11  client__x1_Sector_Operativo   

FIN